In [100]:
import numpy as np
import pandas as pd
import os
import datetime
import warnings
import time
# 遍历所有文件

warnings.filterwarnings('ignore') 
CURRENT_DATE = pd.to_datetime(datetime.date.today())
from tool import *

In [101]:

source_t = r"C:\Users\vkad\Desktop\5-6\243.xlsx"
path = r"C:\Users\vkad\Desktop\5-6\t"
save_dir = r'C:\Users\vkad\Desktop\result'

In [3]:
while True:
    print('如果直接回车日期为当日')
    in_today = input('请输入日期->').strip()
    if in_today:
        try:
            today = pd.to_datetime(in_today)
            break
        except:
            print('请输入正确的日期格式:')
    else:
        today = CURRENT_DATE
        break

如果直接回车日期为当日
请输入日期->


In [6]:
df_yj = pd.read_excel(source_t)

In [9]:
df_yj.replace({"楼栋":{"/":""}},inplace=True)

In [15]:
df_mx = read_mx(path)

读取梅溪郡商业签约明细
合并梅溪郡商业签约明细
读取梅溪郡签约明细
合并梅溪郡签约明细
读取梅溪郡车位明细
合并梅溪郡车位明细

读取滨江商业签约明细
->无滨江商业签约明细表
读取滨江签约明细
合并滨江签约明细
读取滨江车位明细
合并滨江车位明细

读取白鹭郡商业签约明细
合并白鹭郡商业签约明细
读取白鹭郡签约明细
合并白鹭郡签约明细
读取白鹭郡车位明细
合并白鹭郡车位明细

读取百汇商业签约明细
合并百汇商业签约明细
读取百汇签约明细
合并百汇签约明细
读取百汇车位明细
合并百汇车位明细

读取紫台商业签约明细
合并紫台商业签约明细
读取紫台签约明细
合并紫台签约明细
读取紫台车位明细
合并紫台车位明细

读取缇香商业签约明细
合并缇香商业签约明细
读取缇香签约明细
合并缇香签约明细
读取缇香车位明细
合并缇香车位明细

明细表读取完毕


In [23]:
data = df_mx[df_mx['项目'] == "滨江"]
data= data[data['类型']=="车位"]
data[data['姓名'] == "杜文涛"]['房号']

13682    190
Name: 房号, dtype: object

In [102]:
start_t = pd.to_datetime("2019-1-1")
targt_t = pd.to_datetime("2019-4-30")
data = get_interval_data(df_mx,"签约日期",targt_t,start_t)


In [103]:
xm_map_allname  = {
    "滨江":"长沙金域滨江", "百汇":"长沙金域国际", "缇香":"长沙金域缇香"
}

data['产出归属日期'] = pd.to_datetime(data['签约日期']).apply(lambda x:x.strftime("%Y%m"))+"01"
data['归属区域'] = '长沙'
data['事业部'] = "长沙签约事业部" 
data['项目名称'] = data['项目'].copy()
data['开发商名称'] = "万科"
data.replace({"项目名称":xm_map_allname},inplace=True)
data['业务类型'] = data['类型'].copy()


data.replace({"业务类型":{"住宅":"住宅/商业:","商业":"住宅/商业:"}},inplace=True)



data['贷款银行'] =  data['付款方式'].copy()

data['付款方式'] = data['付款方式'].fillna("").str.strip()
data['付款方式'][data['付款方式'].str.contains('^一次性$')] = "一次性付款"
data['付款方式'][data['付款方式'].str.contains('分期')] = "分期付款"
data['付款方式'][data['付款方式'].str.contains('公积金')] = "纯公积金贷款"
data['付款方式'][data['付款方式'].str.contains('组合')] = "组合贷款"


data['付款方式'] [(data['付款方式']!="分期付款") &(data['付款方式']!="") & (data['付款方式']!="一次性付款") & (data['付款方式']!="纯公积金贷款")]  = "商业贷款"
data['业务类型'][data['业务类型'] != '车位'] = data['业务类型'][data['业务类型'] != '车位'] + data['付款方式'][data['业务类型'] != '车位']
data['业务类型'] = data['业务类型'].str.replace("商业贷款|纯公积金贷款","按揭付款")
data = data[data["业务类型"] != "住宅/商业:"]


data['单元'] = "/"
data['物业类型'] = data['类型']
data['物业类型'] = data['物业类型'].str.replace("商业","商铺")
data['套内面积'] = ""
data['业主姓名'] = data['姓名']
data['业主身份证'] = data['身份证号码']
data['电话'] = data['联系电话']
data["签约资料欠缺情况"] = ""
data["贷款资料欠缺情况"] = ""
data["是否关注贴心秘秘"] = "否"

data['客户领走合同日期'] = data['客户契税/维修基金']

data.replace({'贷款银行':{"^一次性|公积金":""}},regex=True,inplace=True)

data['贷款年限'] = ""
data['还款方式'] = ""

data['正式签约日期'] = data["网签日期"]
data['正式签约日期'][data['类型'] == "车位"] = data['签约日期'][data['类型'] == "车位"]

data['正式签约日期'] = pd.to_datetime(data['正式签约日期'])
data = data[data['正式签约日期'].notna()] 


data['非正式签约日期'] = np.nan
data['非正式签约日期'][data['类型'] != "车位"] = data['签约日期'][data['类型'] != "车位"]


data['按揭签约日期'] = data['面签日期']

data['银行送审日期/资料移交银行日期'] = ""
data['银行审批通过日期'] = ""

In [104]:
data["预审批合同移交日期"] = ""
data["备案合同移交日期"] = ""
data["资料齐全日期"] = ""
data["维修基金费用到账日"] = ""
data["资金监管到帐日期"] = ""
data["银行提供备案资料日期"] = ""

data["银行提供预告资料日期"] = ""

data["抵押登记送件日期"] = ""
data["抵押登记出件日期"] = ""

data["完税证明接收日期"] = ""
data["接件日期（贵阳）"] = ""
data["房管局交件日期"] = ""
data["不动产送件日期"] = ""
data["不动产证出件日期"] = ""
data["他项权证送件日期"] = ""
data["他项权证出件日期"] = ""
data["是否需要开票"] = "是"
data["填报备注"] = ""
data["收费方式 "] = ""
data["实际收费日期 "] = ""
data["实际收费金额 "] = ""
data["通知书单号"] = ""
data["预告送件日期"] = ""

data["备案送件日期"] = data[ '送备案时间']
data["备案出件日期"] = data[ '送备案时间']
data["预告出件日期"] = data['出证日期']
data["预告送件日期"] = data["备案完成时间"]

data["银行放款日期/全款到账日"] = data['银行备案资料']
data['成交总价'] = data['买卖合同总价']
fileds = ["产出归属日期", "归属区域", "事业部", "项目名称", "开发商名称", "业务类型", "楼栋", "单元", "房号", "物业类型", "建筑面积", "套内面积", "成交总价", "业主姓名", "业主身份证", "电话", "是否关注贴心秘秘", "签约资料欠缺情况", "贷款资料欠缺情况", "客户领走合同日期", "付款方式", "贷款金额", "贷款银行", "贷款年限", "还款方式", "非正式签约日期", "正式签约日期", "按揭签约日期", "银行送审日期/资料移交银行日期", "银行审批通过日期", "预审批合同移交日期", "备案合同移交日期", "资料齐全日期", "维修基金费用到账日", "资金监管到帐日期", "银行提供备案资料日期", "备案送件日期", "备案出件日期", "银行提供预告资料日期", "预告送件日期", "预告出件日期", "抵押登记送件日期", "抵押登记出件日期", "银行放款日期/全款到账日", "完税证明接收日期", "接件日期（贵阳）", "房管局交件日期", "不动产送件日期", "不动产证出件日期", "他项权证送件日期", "他项权证出件日期", "是否需要开票", "填报备注", "收费方式 ", "实际收费日期 ", "实际收费金额 ", "通知书单号",]

In [106]:
data['楼栋'] = data['楼栋'].fillna("").astype(str)

df_yj['key'] = df_yj['楼栋']+"-"+df_yj['房号']+"-"+df_yj['物业类型']+df_yj["项目名称"]
data['key'] = data['楼栋'] +"-"+data['房号'].astype(str) +"-"+data['物业类型'] + data["项目名称"]
month4_data_set = set(data['key']) - set(df_yj['key'])
data["产出归属日期"][data['key'].isin(month4_data_set)] = "20190401"

In [107]:
writer = pd.ExcelWriter(r"C:\Users\vkad\Desktop\test.xlsx")
data[fileds].to_excel(writer,sheet_name="data",index=False)
writer.save()

In [92]:
set(df_yj['key']) - set(data['key']) 

{'-F2-773-车位长沙金域国际',
 '10-101-住宅紫台',
 '17-2202-住宅长沙金域国际',
 '17-4505-住宅长沙金域国际',
 '17-4702-住宅长沙金域国际',
 '18-1003-住宅长沙金域国际',
 '19-805-住宅长沙金域国际',
 '20-1001-住宅长沙金域国际',
 '20-1203-住宅长沙金域国际',
 '20-2604-住宅长沙金域国际',
 '20-2905-住宅长沙金域国际',
 '4-101-住宅长沙金域国际',
 '4-102-住宅长沙金域国际',
 '5-602-住宅长沙金域滨江',
 '6-105-住宅长沙金域国际',
 '6-106-住宅长沙金域国际',
 '6-1216-住宅长沙金域缇香',
 '8-101-住宅紫台'}

In [91]:
month4_data_set = set(data['key']) - set(df_yj['key'])
month4_data_set

{'-A114-车位长沙金域滨江',
 '-C047-车位长沙金域滨江',
 '-C049-车位长沙金域滨江',
 '-C075-车位长沙金域滨江',
 '-C100-车位长沙金域滨江',
 '-C170-车位长沙金域滨江',
 '-D280-车位长沙金域滨江',
 '-D400-车位长沙金域滨江',
 '-D403-车位长沙金域滨江',
 '-E285-车位长沙金域滨江',
 '-E300-车位长沙金域滨江',
 '-E301-车位长沙金域滨江',
 '-E437-车位长沙金域滨江',
 '-F355-车位长沙金域滨江',
 '-F460-车位长沙金域滨江',
 '-F473-车位长沙金域滨江',
 '-G2-773-车位长沙金域国际',
 '10-204-住宅紫台',
 '10-302-住宅紫台',
 '10-303-住宅紫台',
 '10-304-住宅紫台',
 '10-3801-住宅长沙金域国际',
 '11-202-住宅紫台',
 '11-205-住宅紫台',
 '11-305-住宅紫台',
 '15-101-商铺紫台',
 '15-102-商铺紫台',
 '15-103-商铺紫台',
 '15-104-商铺紫台',
 '15-106-商铺紫台',
 '15-108-商铺紫台',
 '15-109-商铺紫台',
 '15-110-商铺紫台',
 '15-111-商铺紫台',
 '16-1903-住宅长沙金域国际',
 '17-603-住宅长沙金域国际',
 '18-3405-住宅长沙金域国际',
 '20-1604-住宅长沙金域国际',
 '20-3504-住宅长沙金域国际',
 '5-1802-住宅长沙金域滨江',
 '5-501-住宅长沙金域国际',
 '6-105-商铺长沙金域国际',
 '6-106-商铺长沙金域国际',
 '7-1002-住宅紫台',
 '7-1004-住宅紫台',
 '7-1204-住宅紫台',
 '7-1403-住宅紫台',
 '7-1601-住宅紫台',
 '7-2204-住宅紫台',
 '7-2601-住宅紫台',
 '7-2801-住宅紫台',
 '7-602-住宅紫台',
 '7-604-住宅紫台',
 '8-206-住宅紫台',
 '9-404-住宅紫台',
 'S4-101-商铺长沙金域国际',
 'S4-102-